In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df = pd.read_csv('result.csv')
df

,cycle,ambient_temperature,datetime,capacity,voltage_measured,current_measured,temperature_measured,current_load,voltage_load,time,soh_calculated
0,1,24,2008-04-02 15:25:41,1.856487,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000,1.000000
1,1,24,2008-04-02 15:25:41,1.856487,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781,1.000000
2,1,24,2008-04-02 15:25:41,1.856487,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703,1.000000
3,1,24,2008-04-02 15:25:41,1.856487,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781,1.000000
4,1,24,2008-04-02 15:25:41,1.856487,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
185716,132,24,2008-08-20 08:37:19,1.341051,3.443760,-0.002426,35.383979,0.0006,0.000,2686.359,0.722937
185717,132,24,2008-08-20 08:37:19,1.341051,3.453271,-0.000981,35.179732,0.0006,0.000,2700.546,0.722937
185718,132,24,2008-08-20 08:37:19,1.341051,3.461963,0.000209,34.977000,0.0006,0.000,2714.640,0.722937
185719,132,24,2008-08-20 08:37:19,1.341051,3.469907,0.001516,34.785943,0.0006,0.000,2728.750,0.722937


In [21]:
df.drop(['current_load', 'voltage_load', 'cycle', 'ambient_temperature',
         'datetime', 'capacity' ], axis=1, inplace= True)
df.describe

<bound method NDFrame.describe of         voltage_measured  current_measured  temperature_measured      time  \
0               4.191492         -0.004902             24.330034     0.000   
1               4.190749         -0.001478             24.325993    16.781   
2               3.974871         -2.012528             24.389085    35.703   
3               3.951717         -2.013979             24.544752    53.781   
4               3.934352         -2.011144             24.731385    71.922   
...                  ...               ...                   ...       ...   
185716          3.443760         -0.002426             35.383979  2686.359   
185717          3.453271         -0.000981             35.179732  2700.546   
185718          3.461963          0.000209             34.977000  2714.640   
185719          3.469907          0.001516             34.785943  2728.750   
185720          3.477277         -0.001940             34.581660  2742.843   

        soh_calculated  
0   

In [22]:
input_para = ['time', 'voltage_measured', 'current_measured', 'temperature_measured'  ]
train_dataset = df[input_para]
# sc = MinMaxScaler(feature_range=(0,1))
# train_dataset = sc.fit_transform(train_dataset)
print(train_dataset.shape)

(185721, 4)


In [23]:
train_dataset

,time,voltage_measured,current_measured,temperature_measured
0,0.000,4.191492,-0.004902,24.330034
1,16.781,4.190749,-0.001478,24.325993
2,35.703,3.974871,-2.012528,24.389085
3,53.781,3.951717,-2.013979,24.544752
4,71.922,3.934352,-2.011144,24.731385
...,...,...,...,...
185716,2686.359,3.443760,-0.002426,35.383979
185717,2700.546,3.453271,-0.000981,35.179732
185718,2714.640,3.461963,0.000209,34.977000
185719,2728.750,3.469907,0.001516,34.785943


In [24]:
output_var = df['soh_calculated'] 
print(output_var.shape)

(185721,)


In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,  y_train, y_test = train_test_split(train_dataset, output_var, test_size= 0.20)

In [27]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(148576, 4)
(37145, 4)
(148576,)
(37145,)


In [28]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=30)

In [29]:
history = model.fit(x_train, y_train)

In [30]:
soh_pred = model.predict(x_test)
print(soh_pred.shape)
print(soh_pred)

(37145,)
[0.90490924 0.87710327 0.82092679 ... 0.90256091 0.98333657 0.72714954]


In [31]:
y_test

117655    0.906358
122501    0.867834
25239     0.820176
159907    0.922826
2604      0.976813
            ...   
137114    0.800391
102181    0.988704
16374     0.907576
2542      0.976813
86254     0.690375
Name: soh_calculated, Length: 37145, dtype: float64

In [32]:
df_pred = pd.DataFrame( soh_pred , y_test)
df_pred.reset_index(inplace=True)

In [33]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37145 entries, 0 to 37144
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   soh_calculated  37145 non-null  float64
 1   0               37145 non-null  float64
dtypes: float64(2)
memory usage: 580.5 KB


In [34]:
df_pred.head()

,soh_calculated,0
0,0.906358,0.904909
1,0.867834,0.877103
2,0.820176,0.820927
3,0.922826,0.914792
4,0.976813,0.977913


In [35]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score
print('R2 score:', r2_score(y_test, soh_pred))
print('MAE:', mean_absolute_error(y_test, soh_pred))
print('MSE:', mean_squared_error(y_test, soh_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, soh_pred)))

R2 score: 0.982336228073427
MAE: 0.007419293486804705
MSE: 0.0002014299408847106
RMSE: 0.014192601624956244


In [36]:
import pickle
pickle_out = open('soh_rf_est.pkl', 'wb')
pickle.dump(model, pickle_out)
pickle_out.close()

In [17]:
model.predict([[2000, 3.754, -2.111, 24.3 ]])

array([0.95955236])